In [1]:
import torch
import config
import utils
from model import FastText
from sklearn import preprocessing
import numpy as np
from model import FastText

In [2]:
all_label = np.load(
    'model/post_classifier_v1_label_enc.npy', allow_pickle=True)
all_vocab = np.load(
    'model/post_classifier_v1_vocab_enc.npy', allow_pickle=True)
label_enc = preprocessing.LabelEncoder()
vocab_enc = preprocessing.OrdinalEncoder(
    handle_unknown='use_encoded_value', unknown_value=len(all_vocab[0]), dtype=np.int)
vocab_enc.categories_ = all_vocab
label_enc.classes_ = all_label

In [3]:
padding_enc = int(vocab_enc.transform([[config.PAD]]).item())
model = FastText(
    num_classes=len(label_enc.classes_),
    num_vocab=len(vocab_enc.categories_[0])+1,
    padding_enc=padding_enc
)
model.load_state_dict(torch.load('model/post_classifier_v1.pth', map_location=torch.device('cpu')))
model.eval()

FastText(
  (embedding1): Embedding(2000, 300, padding_idx=23)
  (embedding2): Embedding(250499, 300)
  (embedding3): Embedding(250499, 300)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=900, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=5, bias=True)
)

In [7]:
s = '彩妆选购#粉底选择'
words, bigrams, trigrams = utils.encode_sentence(s, vocab_enc)
words = torch.tensor(words.reshape(1, -1))
bigrams = torch.tensor(bigrams.reshape(1, -1))
trigrams = torch.tensor(trigrams.reshape(1, -1))

In [10]:
result, _ = model(words=words, bigrams=bigrams, trigrams=trigrams)
result = torch.argmax(result, 1).detach().cpu().numpy()

array([1], dtype=int64)

In [9]:
label_enc.inverse_transform(result)

array(['彩妆'], dtype=object)